In [ ]:
import pandas as pd
from collections import defaultdict, deque

# Load CSV files into DataFrames
movies_df = pd.read_csv('/content/movies.csv')
stars_df = pd.read_csv('/content/stars.csv')
people_df = pd.read_csv('/content/people.csv')

# Build the graph
graph = defaultdict(set)
movies = defaultdict(set)

# Map movie IDs to actors and vice versa
for _, row in stars_df.iterrows():
    actor_id = row['person_id']
    movie_id = row['movie_id']
    graph[actor_id].add(movie_id)
    movies[movie_id].add(actor_id)

# Convert person IDs to names
person_id_to_name = people_df.set_index('id')['name'].to_dict()
name_to_person_id = {name: id for id, name in person_id_to_name.items()}


In [ ]:
def bfs_shortest_path(start_id, goal_id):
    queue = deque([(start_id, [start_id])])
    visited = set()

    while queue:
        node, path = queue.popleft()
        visited.add(node)

        if node == goal_id:
            return path

        for movie_id in graph[node]:
            for neighbor in movies[movie_id]:
                if neighbor not in visited:
                    queue.append((neighbor, path + [neighbor]))
                    visited.add(neighbor)

    return None

# Example usage
start_name = 'Tom Cruise'
goal_name = 'Cary Elwes'

# Convert names to IDs
start_actor_id = name_to_person_id[start_name]
kevin_bacon_id = name_to_person_id[goal_name]

path_ids = bfs_shortest_path(start_actor_id, kevin_bacon_id)

# Convert path from IDs to names
if path_ids:
    path_names = [person_id_to_name[id] for id in path_ids]
    print("Shortest path using BFS:", " -> ".join(path_names))
else:
    print("No path found using BFS.")


Shortest path using BFS: Tom Cruise -> Kevin Bacon -> Gary Sinise -> Robin Wright -> Cary Elwes


In [ ]:
def dfs_paths(start_id, goal_id):
    stack = [(start_id, [start_id])]
    visited = set()
    paths = []

    while stack:
        node, path = stack.pop()

        if node in visited:
            continue
        visited.add(node)

        if node == goal_id:
            paths.append(path)
            continue

        for movie_id in graph[node]:
            for neighbor in movies[movie_id]:
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor]))

    return paths

# Example usage
paths_ids = dfs_paths(start_actor_id, kevin_bacon_id)

# Convert paths from IDs to names
if paths_ids:
    for i, path_ids in enumerate(paths_ids):
        path_names = [person_id_to_name[id] for id in path_ids]
        print(f"Path {i+1} using DFS:", " -> ".join(path_names))
else:
    print("No paths found using DFS.")


Path 1 using DFS: Tom Cruise -> Kevin Bacon -> Tom Hanks -> Robin Wright -> Mandy Patinkin -> Chris Sarandon -> Cary Elwes
